In [1]:
import os
from data import SQLData
from _decimal import Decimal
from dotenv import load_dotenv

from datasets import DatasetDict, Dataset
#from predict import SQLPredict

# Load environment variables from .env file
load_dotenv("../.env")

# Now you can get the loaded environment variable using os.environ
GITHUB_GIST_TOKEN = os.environ.get("GITHUB_GIST_TOKEN")
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")

In [2]:
name = 'b-mc2/sql-create-context'

In [3]:
sq = SQLData.from_sql_create_context()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
for i in range(len(sq.data[name]['train'])):

    answer = sq.data[name]['train'][i]['answer']

    if answer == 'SELECT artist FROM table_name_29 WHERE mintage > 34 OFFSET 135': 
        print(answer)
        print(sq.data[name]['train'][i])

SELECT artist FROM table_name_29 WHERE mintage > 34 OFFSET 135
{'context': 'CREATE TABLE table_name_29 (artist VARCHAR, mintage INTEGER)', 'question': 'What artist has a mintage of greater than 34,135?', 'answer': 'SELECT artist FROM table_name_29 WHERE mintage > 34 OFFSET 135'}


In [ ]:
sq.preprocess_data(dataset_name=name)

In [ ]:
sq.filter_data(dataset_name=name)

In [ ]:
sq.data[name]

In [ ]:
sq.train_test_split(dataset_name=name, new_dataset_name="data_llama_13b_1_0_0")

In [ ]:
sq.data["data_llama_13b_1_0_0"]['test'][2]

In [ ]:
training_resp = sq.upload_jsonl_gist(
    dataset_name="data_llama_13b_1_0_0",
    token=GITHUB_GIST_TOKEN,
    filename="training_data_llama_13b_1_0_0.jsonl",
    description="Training data for the first version of the llama_13b (1.0.0) model",
)
training_resp['files']['training_data_llama_13b_1_0_0.jsonl']['raw_url']

In [ ]:
testing_resp = sq.upload_jsonl_gist(
    dataset_name="data_llama_13b_1_0_0",
    token=GITHUB_GIST_TOKEN,
    filename="testing_data_llama_13b_1_0_0.jsonl",
    dataset_type="test",
    description="Testing data for the first version of the llama_13b (1.0.0) model",
)
testing_resp['files']['testing_data_llama_13b_1_0_0.jsonl']['raw_url']

In [ ]:
sq.data["data_llama_13b_1_0_0"]

In [ ]:
sq.data["data_llama_13b_1_0_0"].save_to_disk("data_llama_13b_1_0_0")

In [ ]:
import os
import re
import time
import json
import sqlglot
import replicate
import datasets
from replicate import Client as rc
from _decimal import Decimal
from dotenv import load_dotenv
from datasets import DatasetDict, Dataset
from sqlglot.executor import execute

from data import SQLData
from predict import SQLPredict
from eval import SQLEval

from sqlglot.errors import (
    ExecuteError,
    TokenError,
    SchemaError,
    ExecuteError,
    ParseError,
    UnsupportedError,
    SqlglotError,
    OptimizeError,
)

# Load environment variables from .env file
load_dotenv("../.env")

# Now you can get the loaded environment variable using os.environ
GITHUB_GIST_TOKEN = os.environ.get("GITHUB_GIST_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")
REPLICATE_LLAMA_7B_TUNED = os.environ.get("REPLICATE_LLAMA_7B_TUNED")

In [ ]:
# sqp = SQLPredict(OPENAI_API_KEY, REPLICATE_API_TOKEN)
sqp = SQLPredict.from_replicate_model(
    openai_api_key=OPENAI_API_KEY,
    replicate_api_key=REPLICATE_API_TOKEN,
    model_name="llama_2_13b_sql",
    model_id=REPLICATE_LLAMA_7B_TUNED
)
sqd = SQLData.from_sql_create_context()
sqe = SQLEval()

In [ ]:
sqd

In [ ]:
sqd.data["data_llama_13b_1_0_0"] = DatasetDict.load_from_disk("../local_data/data_llama_13b_1_0_0/")
sqd.data["data_llama_13b_1_0_0"]

In [ ]:
rich_testing = sqd.data["data_llama_13b_1_0_0"]['test'].filter(lambda example: example['query_result'] != '[]')

In [ ]:
rich_testing

In [ ]:
rich_testing = rich_testing.filter(lambda example: example['query_result'] != '[(0,)]' and example['query_result'] != '[(None,)]')

In [ ]:
rich_testing.filter(lambda example: example['query_result'] == '[(None,)]')

In [ ]:
rich_testing[6]

In [ ]:
# rich_testing.save_to_disk("rich_testing_llama_13b_1_0_0")

In [ ]:
rich_testing = Dataset.load_from_disk("../local_data/rich_testing_llama_13b_1_0_0")

In [ ]:
rich_testing = rich_testing.filter(lambda example: example['query_result'] != '[(0,)]' and example['query_result'] != '[(None,)]' and example['query_result'] != '[]')

In [ ]:
rich_testing_subset_0_100 = rich_testing.select(range(0, 100))
rich_testing_subset_100_200 = rich_testing.select(range(100, 200))
rich_testing_subset_200_300 = rich_testing.select(range(200, 300))

In [ ]:
# rich_testing_subset = datasets.concatenate_datasets([rich_testing_subset_0_100, rich_testing_subset_100_200, rich_testing_subset_200_300])

In [ ]:
rich_testing_subset_0_100 = rich_testing_subset_0_100.map(sqp.openai_dataset_request)
time.sleep(10)

In [ ]:
rich_testing_subset_100_200 = rich_testing_subset_100_200.map(sqp.openai_dataset_request)
time.sleep(10)

In [ ]:
# rich_testing_subset = rich_testing_subset.map(sqp.openai_dataset_request)
rich_testing_subset_200_300 = rich_testing_subset_200_300.map(sqp.openai_dataset_request)

In [ ]:
#rich_testing_subset = rich_testing_subset.map(sqe.validate_openai_query)
rich_testing_subset_0_100 = rich_testing_subset_0_100.map(sqe.validate_openai_query)
rich_testing_subset_100_200 = rich_testing_subset_100_200.map(sqe.validate_openai_query)
rich_testing_subset_200_300 = rich_testing_subset_200_300.map(sqe.validate_openai_query)

In [ ]:
#rich_testing_subset = rich_testing_subset.map(sqd.format_tuning_data)

rich_testing_subset_0_100 = rich_testing_subset_0_100.map(sqd.format_tuning_data)
rich_testing_subset_100_200 = rich_testing_subset_100_200.map(sqd.format_tuning_data)
rich_testing_subset_200_300 = rich_testing_subset_200_300.map(sqd.format_tuning_data)

In [ ]:
rich_testing_subset_0_100 = rich_testing_subset_0_100.map(sqp.replicate_dataset_request)
time.sleep(10)

In [ ]:
rich_testing_subset_100_200 = rich_testing_subset_100_200.map(sqp.replicate_dataset_request)
time.sleep(10)

In [ ]:
rich_testing_subset_200_300 = rich_testing_subset_200_300.map(sqp.replicate_dataset_request)

In [ ]:
rich_testing_subset = datasets.concatenate_datasets([rich_testing_subset_0_100, rich_testing_subset_100_200, rich_testing_subset_200_300])

In [ ]:
rich_testing_subset.save_to_disk("rich_testing_subset_llama_13b_1_0_0_inferences_two")

In [ ]:
rich_testing_subset[1]

In [ ]:
load_test = Dataset.load_from_disk("rich_testing_subset_llama_13b_1_0_0_inferences")

In [ ]:
load_test[0]

In [ ]:
rich_testing_subset[0]

In [ ]:
rich_testing_subset[0]['tuning_format']

In [ ]:
json.loads(rich_testing_subset[0]['tuning_format'])['prompt']

In [ ]:
rep_client = rc(api_token=REPLICATE_API_TOKEN)

In [ ]:
output = rep_client.run(
    "denverbaumgartner/llama-2-13b-sql:6318cb7d0fce8e4b19262e8aa946a53781d7ca9a56ba8e5c6afc067074d33902",
    input={"prompt": "hello, world"},
)
# The denverbaumgartner/llama-2-13b-sql model can stream output as it's running.
# The predict method returns an iterator, and you can iterate over that output.
for item in output:
    # https://replicate.com/denverbaumgartner/llama-2-13b-sql/versions/6318cb7d0fce8e4b19262e8aa946a53781d7ca9a56ba8e5c6afc067074d33902/api#output-schema
    print(item, end="")

In [ ]:
output = rc.run(
    "denverbaumgartner/llama-2-13b-sql:6318cb7d0fce8e4b19262e8aa946a53781d7ca9a56ba8e5c6afc067074d33902",
    input={"prompt": "hello, world"},
)
# The denverbaumgartner/llama-2-13b-sql model can stream output as it's running.
# The predict method returns an iterator, and you can iterate over that output.
for item in output:
    # https://replicate.com/denverbaumgartner/llama-2-13b-sql/versions/6318cb7d0fce8e4b19262e8aa946a53781d7ca9a56ba8e5c6afc067074d33902/api#output-schema
    print(item, end="")

In [ ]:
output = replicate.run(
    "denverbaumgartner/llama-2-13b-sql:6318cb7d0fce8e4b19262e8aa946a53781d7ca9a56ba8e5c6afc067074d33902",
    input={"prompt": "[INST] <<SYS>>\nContext contains the relevant SQL tables, provide the query that answers the Question in response.\n<</SYS>>\n\nContext: CREATE TABLE table_211714_2 (rank VARCHAR, viewers__in_millions_ VARCHAR)\n\nQuestion: Name the rank for 14.80 viewers[/INST]\n\n"},
)

In [ ]:
output_string = ''.join(item for item in output)

In [ ]:
output_string

In [ ]:
pattern = r"SELECT.*?\n\n"
matches = re.findall(pattern, output_string, re.DOTALL)

In [ ]:
for item in output:
    # https://replicate.com/denverbaumgartner/llama-2-13b-sql/versions/6318cb7d0fce8e4b19262e8aa946a53781d7ca9a56ba8e5c6afc067074d33902/api#output-schema
    print(item, end="")

In [ ]:
for match in matches:
    print(match)

# Model Eval

In [ ]:
import os
import re
import time
import json
import sqlglot
import replicate
import datasets
from replicate import Client as rc
from _decimal import Decimal
from dotenv import load_dotenv
from datasets import DatasetDict, Dataset
from sqlglot.executor import execute

from data import SQLData
from predict import SQLPredict
from eval import SQLEval

from sqlglot.errors import (
    ExecuteError,
    TokenError,
    SchemaError,
    ExecuteError,
    ParseError,
    UnsupportedError,
    SqlglotError,
    OptimizeError,
)

# Load environment variables from .env file
load_dotenv("../.env")

# Now you can get the loaded environment variable using os.environ
GITHUB_GIST_TOKEN = os.environ.get("GITHUB_GIST_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")
REPLICATE_LLAMA_7B_TUNED = os.environ.get("REPLICATE_LLAMA_7B_TUNED")

# sqp = SQLPredict(OPENAI_API_KEY, REPLICATE_API_TOKEN)
sqp = SQLPredict.from_replicate_model(
    openai_api_key=OPENAI_API_KEY,
    replicate_api_key=REPLICATE_API_TOKEN,
    model_name="llama_2_13b_sql",
    model_id=REPLICATE_LLAMA_7B_TUNED
)
sqd = SQLData.from_sql_create_context()
sqe = SQLEval()

In [ ]:
inf_data = Dataset.load_from_disk("../local_data/rich_testing_subset_llama_13b_1_0_0_inferences")

In [ ]:
inf_data[0]

In [ ]:
test = inf_data.map(sqe.validate_replicate_query)

In [ ]:
test[0]

In [ ]:
test.filter(lambda example: example['replicate_valid'] == True)

In [ ]:
test.filter(lambda example: example['openai_valid'] == True)

In [ ]:
rep_wrong = test.filter(lambda example: example['replicate_valid'] == False)

In [ ]:
resp = rep_wrong[0]['replicate_inference']

In [ ]:
resp

In [ ]:
pattern = r"SELECT.*?\n\n"
matches = re.findall(pattern, resp, re.DOTALL)

In [ ]:
def replicate_response_parser(dataset):
    
    replicate_inference = dataset['replicate_inference']
    replicate_result = dataset['replicate_result']
    
    tables = json.loads(dataset['filler_data'])
    
    pattern = r"SELECT.*?(?=\n|\[/|,\[INST\])"
    matches = re.findall(pattern, replicate_inference, re.DOTALL)
    
    result = None
    valid_result = None 
    valid_statement = None
    
    for match in matches:
        try: 
            result = execute(match, tables=tables)
            if result.rows is not None: 
                valid_result = str(result.rows)
                valid_statement = match
        except:
            pass

    if valid_result and valid_statement: 
        dataset['replicate_inference'] = valid_statement 
        dataset['replicate_result'] = valid_result 
        dataset['replicate_valid'] = True
    else: 
        dataset['replicate_inference'] = replicate_inference 
        dataset['replicate_result'] = replicate_result
        dataset['replicate_valid'] = False
    return dataset

    

In [ ]:
rep_wrong_c = rep_wrong.map(replicate_response_parser)

In [ ]:
rep_wrong_b = rep_wrong_c.filter(lambda example: example['replicate_valid'] == False)

In [ ]:
rep_wrong_b

In [ ]:
rep_wrong_b[5]

In [ ]:
rep_wrong_b[0]['replicate_inference'].replace('Query:', '')

In [ ]:
query = rep_wrong_c[0]['replicate_inference']
tables = json.loads(rep_wrong_c[0]['filler_data'])

In [ ]:
result = execute(query, tables=tables)

In [ ]:
result

In [ ]:
def remove_newline_query(dataset):

    substrings_to_replace = ["```", "\nQuery:", "\n", "SQL>"]
    query_string = dataset["replicate_inference"]

    for substring in substrings_to_replace:
        query_string = query_string.replace(substring, "")

    return {"replicate_inference": query_string}
